In [3]:
CREATE_CSV_FILES = True

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import librosa
import csv
import os

In [7]:
if (CREATE_CSV_FILES == True):
    extractWavFeatures("./train", TRAIN_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")

The features of the files in the folder ./train will be saved to train.csv
CSV Header:  ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'label']
End of extractWavFeatures
CSV files are created


In [10]:
#Reading a dataset and convert file name to corresbonding umnber

import pandas as pd
import csv
from sklearn import preprocessing

def preProcessData(csvFileName):
    print(csvFileName+ " will be preprocessed")
    data = pd.read_csv(csvFileName)
    # we have six speakers: 
    # 0: Jackson
    # 1: Nicolas 

    filenameArray = data['filename'] 
    speakerArray = []
    #print(filenameArray)
    for i in range(len(filenameArray)):
        speaker = filenameArray[i][1]
        #print(speaker)
        if speaker == "e":
            speaker = "0"
        elif speaker == "o":
            speaker = "1"
        #print(speaker)
        speakerArray.append(speaker)
    data['number'] = speakerArray
    #Dropping unnecessary columns
    data = data.drop(['filename'],axis=1)
    data = data.drop(['label'],axis=1)
    data = data.drop(['chroma_stft'],axis=1)
    data.shape

    print("Preprocessing is finished")
    print(data.head())
    data.to_csv("train1.csv")
    return data

trainData = preProcessData(TRAIN_CSV_FILE)

train.csv will be preprocessed
Preprocessing is finished
       rmse  spectral_centroid  spectral_bandwidth      rolloff  \
0  0.092040        1321.000305         1514.088772  2604.666944   
1  0.040131        2066.606647         1826.492715  3700.532419   
2  0.087739        1358.459739         1569.512437  2675.561248   
3  0.067752        1654.466786         1676.890568  3089.722023   
4  0.057248        1718.945521         1683.446906  3118.167243   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3      mfcc4     mfcc5  \
0            0.062401 -239.876511  151.356308 -19.939732  40.241650  8.041357   
1            0.146110 -370.623779   93.204094   0.186144  15.445115  4.331242   
2            0.063696 -246.876663  143.090683 -17.492460  33.518051  7.008014   
3            0.083799 -312.921753   92.578491   0.316432  17.647921  5.095498   
4            0.096406 -346.985474  101.074364  -5.108944  17.567461  1.459630   

   ...    mfcc12    mfcc13     mfcc14    mfcc15    mf

In [19]:
# Splitting the dataset into training, validation and testing dataset
from sklearn.model_selection import train_test_split
X = np.array(trainData.iloc[:, :-1], dtype = float)
X_train, X_val = train_test_split(X, test_size=0.3, random_state=42)


print("X from training data:", X_train.shape)
print("X from validation data:", X_val.shape)

X from training data: (42, 25)
X from validation data: (18, 25)


In [20]:
    #Normalizing the dataset
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()
X_train = scaler.fit_transform( X_train )
X_val = scaler.transform( X_val )

print("X from training data", X_train.shape , X_train)
print("X from validation data", X_val.shape,X_val)

X from training data (42, 25) [[ 0.55960112 -0.61642897 -0.42705102 ...  0.49357835 -0.85115465
   1.02111041]
 [-0.82499108  0.18937035  0.10926744 ... -1.84285258  1.22274233
  -1.27900258]
 [-0.60241978  0.12787893  0.01765042 ... -0.56061286  0.67841375
  -0.8158168 ]
 ...
 [-1.68840183  1.79693836  1.88384986 ... -0.25143553  0.45000776
   1.18813942]
 [-0.02725628 -0.24763454 -0.37672624 ... -0.62529488 -0.25383276
  -1.16842333]
 [-0.48076675  0.2827126  -0.07804563 ... -1.08214726  1.36165427
  -1.16511362]]
X from validation data (18, 25) [[ 6.65097583e-01 -6.63890623e-01 -7.37773006e-01 -6.01765002e-01
  -5.82549315e-01  8.57332003e-01  1.04125546e+00 -1.14558441e+00
   1.27775903e+00  6.18258289e-01 -1.27397407e+00  1.48124104e-01
  -2.09700291e-01 -1.07899634e+00  1.50521039e-01  9.15076389e-01
  -2.64114394e-01  5.55319153e-01  2.45940937e-01  1.53842896e-01
  -8.15737238e-01 -1.21604286e+00  7.61614061e-01 -4.28576368e-01
  -7.82017710e-02]
 [-1.44759500e+00  3.93839045e-

In [30]:
#Creating a Model
from keras import models
from keras import layers
import keras

# model 1
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Learning Process of a model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# simple early stopping
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)


In [33]:
#Train with early stopping to avoid overfitting
history = model.fit(X_train,
                    validation_data=(X_val),
                    epochs=50,
                    batch_size=32, 
                    callbacks=[es])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()